In [3]:
# pip install requests pandas
import requests, pandas as pd
start, end = "2000-01-01", "2025-09-23"                       # ~50y
# Example: get observations for a known series name (replace SERIES_ID with actual)
url = f"https://www.bankofcanada.ca/valet/observations/SERIES_ID/csv?start_date={start}&end_date={end}"
r = requests.get(url)
r.raise_for_status()
df = pd.read_csv(pd.compat.StringIO(r.text))
print(df.head(), df.shape)

HTTPError: 404 Client Error: Not Found for url: https://www.bankofcanada.ca/valet/observations/SERIES_ID/csv?start_date=2000-01-01&end_date=2025-09-23

In [4]:
import requests, pandas as pd
from io import StringIO

start, end = "1975-01-01", "2025-09-23"
series_id = "V39079"   # target for the overnight rate

url = f"https://www.bankofcanada.ca/valet/observations/{series_id}/csv?start_date={start}&end_date={end}"
r = requests.get(url)
r.raise_for_status()

df = pd.read_csv(StringIO(r.text))
print(df.head(), df.shape)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 5, saw 3


In [2]:
import pandas as pd
# Example FRED series id from search: IRSTCB01CAQ156N
url = "https://fred.stlouisfed.org/graph/fredgraph.csv?id=IRSTCB01CAQ156N"
df = pd.read_csv(url, parse_dates=["DATE"])
df.rename(columns={"DATE":"date","IRSTCB01CAQ156N":"rate"}, inplace=True)
df = df.set_index("date").sort_index()
print(df.first_valid_index(), df.last_valid_index(), df.head())

ValueError: Missing column provided to 'parse_dates': 'DATE'

In [5]:
import requests, pandas as pd
from datetime import datetime

def valet_json(series_id, start="1975-01-01", end=None):
    end = end or datetime.today().date().isoformat()
    url = f"https://www.bankofcanada.ca/valet/observations/{series_id}/json?start_date={start}&end_date={end}"
    obs = requests.get(url).json()["observations"]
    df = pd.DataFrame(obs)
    # columns look like: "d" (date) and "{series_id}" as strings
    df = df.rename(columns={"d":"date"})
    df["date"] = pd.to_datetime(df["date"])
    df[series_id] = pd.to_numeric(df[series_id], errors="coerce")
    return df[["date", series_id]].sort_values("date").reset_index(drop=True)

# Policy rate (overnight target; formal series since 1994)
overnight = valet_json("V39079", start="1975-01-01", end="2025-09-23")
# Bank Rate (older proxy; goes further back)
bankrate  = valet_json("V39078", start="1975-01-01", end="2025-09-23")

# Build the longest continuous policy-rate proxy:
# Prefer V39079 when available; fall back to V39078 earlier.
df = pd.merge(bankrate, overnight, on="date", how="outer").sort_values("date")
df["policy_rate"] = df["V39079"].combine_first(df["V39078"])
df = df[["date", "policy_rate"]].dropna().reset_index(drop=True)

print(df.head(), df.tail(), df.shape)

Empty DataFrame
Columns: [date, policy_rate]
Index: [] Empty DataFrame
Columns: [date, policy_rate]
Index: [] (0, 2)


In [6]:
df

,date,policy_rate


In [7]:
import requests, pandas as pd
from datetime import datetime

def valet_json(series_id, start="1975-01-01", end=None):
    end = end or datetime.today().date().isoformat()
    url = f"https://www.bankofcanada.ca/valet/observations/{series_id}/json?start_date={start}&end_date={end}"
    j = requests.get(url)
    j.raise_for_status()
    obs = j.json().get("observations", [])
    if not obs:
        raise ValueError(f"No observations returned for {series_id}. URL was: {url}")
    df = pd.DataFrame(obs).rename(columns={"d":"date"})
    # Values are often like {"v":"1.75"}; sometimes plain strings.
    def extract_v(x):
        if isinstance(x, dict):
            return x.get("v")
        return x
    df[series_id] = pd.to_numeric(df[series_id].map(extract_v), errors="coerce")
    df["date"] = pd.to_datetime(df["date"])
    return df[["date", series_id]].sort_values("date").reset_index(drop=True)

# Overnight target (policy rate; starts in 1994)
overnight = valet_json("V39079", start="1975-01-01", end="2025-09-23")
# Bank Rate (older proxy; goes further back)
bankrate  = valet_json("V39078", start="1975-01-01", end="2025-09-23")

# Stitch longest proxy: prefer overnight; fallback to bank rate earlier
df = pd.merge(bankrate, overnight, on="date", how="outer").sort_values("date")
df["policy_rate"] = df["V39079"].combine_first(df["V39078"])
df = df[["date", "policy_rate"]].dropna().reset_index(drop=True)

print(df.head(10))
print(df.tail(10))
print(df.shape)

        date  policy_rate
0 2009-04-21         0.25
1 2009-04-22         0.25
2 2009-04-23         0.25
3 2009-04-24         0.25
4 2009-04-27         0.25
5 2009-04-28         0.25
6 2009-04-29         0.25
7 2009-04-30         0.25
8 2009-05-01         0.25
9 2009-05-04         0.25
           date  policy_rate
4272 2025-09-10         2.75
4273 2025-09-11         2.75
4274 2025-09-12         2.75
4275 2025-09-15         2.75
4276 2025-09-16         2.75
4277 2025-09-17         2.75
4278 2025-09-18         2.50
4279 2025-09-19         2.50
4280 2025-09-22         2.50
4281 2025-09-23         2.50
(4282, 2)


In [9]:
import plotly.express as px

fig = px.line(
    df,
    x="date",
    y="policy_rate",
    title="Bank of Canada Policy Rate (Bank Rate + Overnight Target)",
    labels={"date": "Date", "policy_rate": "Rate (%)"}
)
fig.show()

In [18]:
df.head()

,date,policy_rate
0,2009-04-21,0.25
1,2009-04-22,0.25
2,2009-04-23,0.25
3,2009-04-24,0.25
4,2009-04-27,0.25


In [21]:
import numpy as np
import pandas as pd
from datetime import timedelta
import plotly.express as px

# ---- Input: your historical dataframe ----
# df with columns ["date","policy_rate"] as you showed
df = df.copy()
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values("date")

hist = (df.set_index("date")["policy_rate"]
          .asfreq("B")
          .ffill()
          .dropna()
          .astype(float))
dR = hist.diff().dropna()

# Estimate distribution of daily changes ∆R. Two variants:
# (A) use all days (captures rarity of moves) - DEFAULT
# (B) use only non-zero change days (uncomment if you want fatter jumps)
dR = hist.diff().dropna()
# dR_nz = dR[dR != 0]  # <- alt

# Robust sanity check: winsorize extreme outliers a bit (policy data can have announcement-day artifacts)
lo, hi = dR.quantile([0.005, 0.995])
dR_w = dR.clip(lo, hi)

mu  = dR_w.mean()
sig = dR_w.std(ddof=1)

print(f"Estimated daily change μ={mu:.5f} pp/day, σ={sig:.5f} pp/day  (from {len(dR_w)} days)")

# ---- Simulation params ----
n_sims   = 5000
horizon_days = 3 * 365   # ~3 calendar years; we’ll simulate on business days and map to month-end for plotting
rate_min, rate_max = 0.0, 15.0  # clip band for sanity

# Build future business-day index from the last historical date
start = hist.index[-1]
end   = start + pd.Timedelta(days=horizon_days)
future_idx = pd.bdate_range(start=start + pd.Timedelta(days=1), end=end)  # start tomorrow, business days only
T = len(future_idx)

# ---- Monte Carlo: additive noise on the **level** via daily ∆R ~ N(μ, σ²)
last_rate = float(hist.iloc[-1])
# Draw shocks (T x n_sims)
rng = np.random.default_rng(42)
shocks = rng.normal(loc=mu, scale=sig, size=(T, n_sims))

# Accumulate path-wise changes
paths = np.cumsum(shocks, axis=0) + last_rate
# But that would start immediately from last_rate + shock_1; more accurate is:
paths = np.vstack([np.full((1, n_sims), last_rate), last_rate + np.cumsum(shocks, axis=0)])
sim_idx = pd.Index([hist.index[-1]] + list(future_idx), name="date")
paths = pd.DataFrame(paths, index=sim_idx)

# Clip to plausible range (policy doesn’t go negative; also cap extremes)
paths = paths.clip(lower=rate_min, upper=rate_max)

# ---- Convert to **monthly** for nicer plotting (month-end snapshot)
paths_monthly = paths.resample("M").last()

# Summaries: median & 95% CI across sims
median = paths_monthly.median(axis=1).rename("median")
lo95   = paths_monthly.quantile(0.025, axis=1).rename("p2p5")
hi95   = paths_monthly.quantile(0.975, axis=1).rename("p97p5")

summ = pd.concat([median, lo95, hi95], axis=1)
summ.index.name = "date"

# Also include last 5 years of **historical** monthly for context
hist_monthly = hist.resample("M").last().rename("historical")
# hist_tail = hist_monthly.loc[hist_monthly.index.max() - pd.DateOffset(years=5):]
# Show all history
hist_df = hist_monthly.reset_index()

# ---- Plot with Plotly (interactive)
hist_df = hist_tail.reset_index()
proj_df = summ.reset_index()

fig = px.line(hist_df, x="date", y="historical", title="Bank of Canada Policy Rate – MC Projection (Monthly, 95% CI)")
# Add median
fig.add_scatter(x=proj_df["date"], y=proj_df["median"], mode="lines", name="Median (proj)")
# Add CI band (as two traces to create a filled area)
fig.add_scatter(x=proj_df["date"], y=proj_df["p97p5"], mode="lines", name="97.5% (proj)", line=dict(width=0))
fig.add_scatter(x=proj_df["date"], y=proj_df["p2p5"], mode="lines", name="2.5% (proj)",
                fill="tonexty", line=dict(width=0))

fig.update_layout(yaxis_title="Rate (%)", xaxis_title="Date", legend_title=None)
fig.show()

# ---- Quick text summary
last = last_rate
m_12  = float(summ.iloc[min(11, len(summ)-1)]["median"]) if len(summ) else np.nan
m_24  = float(summ.iloc[min(23, len(summ)-1)]["median"]) if len(summ) > 23 else np.nan
m_36  = float(summ.iloc[-1]["median"]) if len(summ) else np.nan
print(f"Last observed: {last:.2f}%. Median projected ~12m: {m_12:.2f}%, ~24m: {m_24:.2f}%, ~36m: {m_36:.2f}%")

Estimated daily change μ=0.00134 pp/day, σ=0.01827 pp/day  (from 4285 days)


Last observed: 2.50%. Median projected ~12m: 2.83%, ~24m: 3.19%, ~36m: 3.57%
